Succinct addressing for list-embed that preserves ancestry information as much as possible is the idea here.

In [1]:
'''
how many elements at the end of a are prefixing b?
'''
def list_end_prefix_len(la,lb):
    ct = 0
    for i,x in enumerate(la):
        if lb[ct] != x:
            ct = 0
        else:
            ct += 1
    return ct

'''
a list of one number <x> reduces to <(<x>,1)>
		a list of two numbers <x,y> reduces to either
        <(<x,y>,1)> if x!=y or <(<x>,2)> if x=y
		a list of n numbers <x0,x1...>, which reduces to <(<x0,..>,r0),...(<xk,..>,ri)>,
        when concatenated with <y> reduces to 
			<(<x0,..>,r0),...(<xk,..,y>,ri)> if ri = 1
			<(<x0,..>,r0),...(<xk,..>[:-j],ri),(<y>,j+1)> if last j elts of the xk list = y
			<(<x0,..>,r0),...(<xk,..>,ri),(<y>,1)> otherwise
			
			this is then followed by a pass through 
            the list (or something smarter, maybe a hashset) 
            to determine if two subsequent lists can be merged 
            (i.e. are the same or the first is a prefix for the second)
'''
def reduce_num_list_rec(l):
    if len(l) == 1:
        return [(l,1)]
    #start with the first two
    x,y = l[:2]
    lr = [[[x,y],1]] if x != y else [[[x],2]]
    #do the rest
    for new_num in l[2:]:
        prev_last = lr[-1][0][-1]
        ri = lr[-1][1]#repetition count for the previous one
        if prev_last == new_num:
            if len(lr[-1][0]) == 1:
                lr[-1][1] += 1
            else:
                del lr[-1][0][-1]
                lr.append([[new_num],2])
        elif ri == 1:
            lr[-1][0].append(new_num)
        else:
            lr.append([[new_num],1])
        
        #pass-through to check for subsequent prefixes
        for i in range(len(lr)-1):
            if lr[i][1] > 1:
                continue#this will probably end up making things less efficient if we do this
            if lr[i+1][1] > 1:
                continue#this can make the string just wrong
            la = lr[i][0]
            lb = lr[i+1][0]
            pref_len = list_end_prefix_len(la,lb)
            if pref_len > 0:
                #shave off those elements of a
                new_elt = la[-pref_len:]
                lr[i][0] = la[:-pref_len]
                #trim b
                lr[i+1][0] = lb[pref_len:]
                #add in the new element
                lr.insert(i+1,[new_elt,2])
                if pref_len == len(la):
                    del lr[i]
                if pref_len == len(lb):
                    del lr[i+2]#which has moved
    return lr

def unreduce_num_list(l):
    lr = []
    for seq,nreps in l:
        lr += [x for _ in range(nreps) for x in seq]
    return lr

In [2]:
reduce_num_list_rec([0,0,0,12,10,101,0,1,2,1,2])

[[[0], 3], [[12, 10, 101, 0, 1, 2, 1, 2], 1]]

so this won't work I think... I have a better idea though

In [3]:
def reduce_num_list_single(l):
	def rsql_eval(rsq_lock,rseq_current_idx,nreps_this,rseq):
		if rsq_lock:
			rsq_lock = False
			if rseq_current_idx != 0:  #less than a full repetition
				if nreps_this > 1:  #we got more than one, but just not an integer number
					lr.append([rseq,nreps_this])
					rseq = rseq[:rseq_current_idx]
					nreps_this = 1
				else:  #we got less than one total -- just carry on as though we'd seen nothing repetitive
					rseq = rseq+rseq[:rseq_current_idx]
			else:
				lr.append([rseq,nreps_this])
				rseq = rseq[:rseq_current_idx]
				nreps_this = 1
		return rsq_lock,rseq_current_idx,nreps_this,rseq

	#just look for repeated sequences
	lr = []
	rseq = [l[0]]
	rseq_current_idx = 0
	nreps_this = 1
	rsq_lock = False
	for i,x in enumerate(l[1:]):
		if (x == rseq[rseq_current_idx]) and (len(l) - i >= len(rseq)):
			rseq_current_idx += 1
			rsq_lock = True
			if rseq_current_idx == len(rseq):
				rseq_current_idx = 0
				nreps_this += 1
		else:
			rsq_lock,rseq_current_idx,nreps_this,rseq = rsql_eval(rsq_lock,rseq_current_idx,nreps_this,rseq)
			rseq.append(x)
			rseq_current_idx = 0
	rsq_lock,rseq_current_idx,nreps_this,rseq = rsql_eval(rsq_lock,rseq_current_idx,nreps_this,rseq)
	if len(rseq) > 0:
		lr.append([rseq,nreps_this])
	return lr

def reduce_num_list(l):
	#do the single thing for each starting point of each list we get
	lr = []
	lq = [l]
	while len(lq) > 0:
		lt = lq.pop(0)
		lrt = reduce_num_list_single(lt)
		if len(lrt[0][0]) == len(lt):#no reductions
			if len(lrt[0][0]) == 1:
				lr.append([lrt[0][0],lrt[0][1]])
			else:
				lr.append([[lrt[0][0][0]],1])
				lq.append(lrt[0][0][1:])
		else:
			#find long, single-repeated lists, add them to the queue
			for rseq,nreps in lrt:
				if (nreps == 1) and (len(rseq) > 1):
					lq.append(rseq)
				else:
					lr.append([rseq,nreps])
	#recombine singletons
	lr_final = [lr[0]]
	for i in range(1,len(lr)):
		if (lr[i][1] == 1) and (lr_final[-1][1] == 1):
			lr_final.append([lr_final[-1][0] + lr[i][0],1])
			del lr_final[-2]#this is i-1
		else:
			lr_final.append(lr[i])
	return lr_final

In [4]:
reduce_num_list([0,0,0,12,10,101,0,1,2,1,2])

[[[0], 3], [[12, 10, 101, 0], 1], [[1, 2], 2]]

In [5]:
import random
lt = [random.randint(0,10) for _ in range(50)]
lt

[0,
 1,
 9,
 1,
 10,
 8,
 9,
 10,
 5,
 8,
 10,
 5,
 9,
 4,
 6,
 7,
 7,
 4,
 3,
 7,
 6,
 8,
 1,
 2,
 7,
 0,
 8,
 9,
 4,
 1,
 6,
 6,
 9,
 7,
 4,
 9,
 4,
 7,
 10,
 7,
 7,
 10,
 4,
 0,
 8,
 7,
 7,
 8,
 1,
 6]

In [6]:
red = reduce_num_list(lt)
unred = unreduce_num_list(red)
assert(unred == lt)
red

[[[0, 1, 9, 1, 10, 8, 9, 10, 5, 8, 10, 5, 9, 4, 6], 1],
 [[7], 2],
 [[4, 3, 7, 6, 8, 1, 2, 7, 0, 8, 9, 4, 1], 1],
 [[6], 2],
 [[9, 7, 4, 9, 4, 7, 10], 1],
 [[7], 2],
 [[10, 4, 0, 8], 1],
 [[7], 2],
 [[8, 1, 6], 1]]

Alright... seems to work. how about the encoding?

In [13]:
rrs_unformatted_all(red)

[[[0, 1, 9, 1, 10, 8, 9, 10, 5, 8, 10, 5, 9, 4, 6], 1],
 [[7], 2],
 [[4, 3, 7, 6, 8, 1, 2, 7, 0, 8, 9, 4, 1], 1],
 [[6], 2],
 [[9, 7, 4, 9, 4, 7, 10], 1],
 [[7], 2],
 [[10, 4, 0, 8], 1],
 [[7], 2],
 [[8, 1, 6], 1]]

In [12]:
rrs_unformatted_all([[[0,1,2,1,0,2,1,2,0,512,0,1,2,1],1]])

[[[0, 1, 2, 1, 0, 2, 1, 2, 0], 1], [[512], 1], [[0, 1, 2, 1], 1]]

In [11]:
'''
unreduction for efficiency's sake
'''

'''
do the rrs unreduction on all eligible lists
'''
def rrs_unformatted_all(l):
    red_lists = [(x[0],i) for i,x in enumerate(l) if (x[1] == 1) and (len(x[0]) > 1)]
    nwidth_w = w(max([w(x) for rsq,_ in l for x in rsq]))
    rsq_w = max([w(len(rsq)) for rsq,_ in l])
    cumulative_ins_shift = 0
    for rl,ins_idx in red_lists:
        rlw = [w(x) for x in rl]
        #reduce
        rls = recursive_redlist_split(rl,nwidth_w,rsq_w,rlw=rlw)
#         print(rls)
#         print()
        if len(rl) > 1:
            #insert the new ones first
            for nrl in reversed(rls):
                l.insert(ins_idx+1+cumulative_ins_shift,[nrl,1])
#                 print(l)
            del l[ins_idx+cumulative_ins_shift]#get rid of the old one
            cumulative_ins_shift += len(rls) - 1
    return l
            

'''
rl should just be a list of numbers -- this should only be called on no-repeat runs
'''
def recursive_redlist_split(rl,nwidth_w,rsq_w,rlw=None):
    if rlw is None:
        rlw = [w(x) for x in rl]
    split_max_val = -float('inf')
    split_max_left = None
    split_max_right = None
    
    for l in range(len(rl)):
        for r in range(l+2,len(rl)):
            run_left = rlw[:l+1]
            run_center = rlw[l+1:r]
            run_right = rlw[r:]
            w_c = max(run_center)
            w_l = max(run_left)
            w_r = max(run_right)
            g_1 = sum([1 if len(x) > 1 else 0 for x in [run_left,run_center,run_right]])
            e_1 = sum([1 if len(x) == 1 else 0 for x in [run_left,run_center,run_right]])
            split_val = (w_c - w_l)*(l+1) + (w_c - w_r)*(len(rl)-r) - ((2 + nwidth_w)*(g_1 + e_1) + rsq_w*g_1)
#             print(w_c,w_l,w_r,g_1,e_1,l,r,split_val)
            if split_val > split_max_val:
                split_max_val = split_val
                split_max_left = l
                split_max_right = r
                
    if split_max_val > 0:
        leftlist = recursive_redlist_split(rl[:split_max_left+1],nwidth_w,rsq_w,rlw[:split_max_left+1])
        centerlist = recursive_redlist_split(rl[split_max_left+1:split_max_right],nwidth_w,rsq_w,rlw[split_max_left+1:split_max_right])
        rightlist = recursive_redlist_split(rl[split_max_right:],nwidth_w,rsq_w,rlw[split_max_right:])
        return leftlist + centerlist + rightlist
    else:
        return [rl]#put it in another list to preserve split locations


In [10]:
import math

def tbw(n,w):#to binary with width
        return bin(n)[2:].zfill(w)
def w(n):
    return len(bin(n)) - 2

'''
5 bits: width of number width (w)
5 bits: repetition width (r)
5 bits: length of repeated sequence width (l)
<given a reduced-length list with repeating number sequences (s')> #how do we do this?
for each (sequence,nreps) in s':
    1 bit: contains any repetition (1) or not (0) <this is set by nreps > 1>
    1 bit: contains multiple numbers (1) or not (0)
    w bits: number width for this sequence (t)
    <if multiple numbers is set>
        l bits: length of repeated sequence
    for each number in sequence:
        t bits: the number
    <if repetition bit is set>
        r bits: nreps
'''
def encode_num_list(l,unreduce=True):
    #reduce the list
    lred = reduce_num_list(l)
    #unreduce the good bits
    if unreduce:
        lred = rrs_unformatted_all(lred)
    nwidth_w = w(max([w(x) for rsq,_ in lred for x in rsq]))
    rep_w = max([w(x) for _,x in lred])
    rsq_w = max([w(len(rsq)) for rsq,_ in lred])
    enc_st = ''.join([tbw(x,5) for x in [nwidth_w,rep_w,rsq_w]])
    
    for seq,nreps in lred:
        if nreps > 1:
            enc_st += '1'
        else:
            enc_st += '0'
        if len(seq) > 1:
            enc_st += '1'
        else:
            enc_st += '0'
        this_numw = max([w(x) for x in seq])
        enc_st += tbw(this_numw,nwidth_w)
        if len(seq) > 1:
            enc_st += tbw(len(seq),rsq_w)
        for num in seq:
            enc_st += tbw(num,this_numw)
        if nreps > 1:
            enc_st += tbw(nreps,rep_w)
            
    return enc_st

In [53]:
encode_num_list([0,0,0,12,10,101,0,1,2,1,2])

[[[0], 3], [[12, 10, 101, 0], 1], [[1, 2], 2]]


'0001100010000111000101101111100000110000010101100101000000011010010011010'

In [14]:
encode_num_list([0,1,2,1,0,2,1,2,0,512,0,1,2,1])

'00100000010010001001010010001100100100110000010101000000000010010010000011001'

In [15]:
encode_num_list([0,1,2,1,0,2,1,2,0,512,0,1,2,1],unreduce=False)

'001000000100100011010111000000000000000000001000000001000000000010000000000000000001000000000010000000010000000000010000000000000000000000000000100000000100000000001'

damn

In [16]:
lt = [random.randint(0,128) for _ in range(500)]
print('unreduced: {:.3f}'.format(len(encode_num_list(lt))/len(lt)))
print('not unreduced: {:.3f}'.format(len(encode_num_list(lt,unreduce=False))/len(lt)))

unreduced: 7.970
not unreduced: 8.012


More precisely we seem to be getting $\Theta(\lg(k^*)N)$ bits per, so like the $N\lg(N)$ scenario we'd thought up before. This isn't really surprising since purely random data *should* be high entropy.

Also it's kind of unsurprising that unreduction makes little difference on data like this -- super high entropy stuff is no more likely to have a wide number next to a little number than it is to have a wide number next to many wide numbers

All that said, let's look at a power law list instead.

In [268]:
addr_len = 500
addr_maxval = 2048 #this is k*
# addr_zeroprob = 0.5 #prob of link in address chain being zero (if we want a real distribution we have to let this be implicitly defined)
addr_decay_rate = 0.99 #how much less likely is number x+1 than x?
addr_zeroprob = (addr_decay_rate-1)/(addr_decay_rate**(addr_maxval+1)-1)
lt_pl_probs = [addr_zeroprob*(addr_decay_rate**x) for x in range(addr_maxval)]
lt_powerlaw = random.choices(list(range(addr_maxval)),weights=lt_pl_probs,k=addr_len)
lt_powerlaw[:20],len(lt_powerlaw)

([5,
  166,
  85,
  7,
  107,
  0,
  185,
  32,
  7,
  30,
  243,
  61,
  211,
  170,
  3,
  28,
  78,
  65,
  50,
  12],
 5000)

In [269]:
unred = encode_num_list(lt_powerlaw)
print('unreduced: {} (ratio: {:.3f})'.format(len(unred),len(unred)/len(lt_powerlaw)))
red = encode_num_list(lt_powerlaw,unreduce=False)
print('not unreduced: {} (ratio: {:.3f})'.format(len(red),len(red)/len(lt_powerlaw)))

if len(unred) > len(red):
    print('MEGAWAT: unreduced resulted in longer output')

unreduced: 46048 (ratio: 9.210)
not unreduced: 48484 (ratio: 9.697)


aight for some reason this sometimes does stupid things (unreduced being longer) which I think is because of some mysterious bug in the code (which I *believe* I've fixed -- it had to do with how new split lists were being inserted) but as a proof of concept I think it works: unreduction significantly reduces the size of inputs in certain distributions (and, in **theory** can't increase them), and our performance is better on powerlaw distributions (but not *much* better :E), as we should expect.

In [242]:
#here's an example of a list that breaks unreduction for some reason
pl_unred_breaker = lt_powerlaw
reduce_num_list(pl_unred_breaker)

[[[5, 0, 6, 7, 8, 0, 37, 10, 15, 4, 0, 7, 11, 2, 14, 0, 3], 1],
 [[4], 2],
 [[20,
   1,
   9,
   23,
   0,
   50,
   2,
   3,
   4,
   2,
   5,
   19,
   20,
   6,
   1,
   9,
   19,
   18,
   6,
   3,
   6,
   1,
   7,
   11,
   0,
   7,
   2,
   19,
   2,
   17,
   2],
  1]]

In [250]:
unreduce_num_list(rrs_unformatted_all(reduce_num_list(pl_unred_breaker))) == pl_unred_breaker

True

Think I fixed it.

## What about combining dyadic and list addressing schemes?

In [17]:
from Embedding import *

In [94]:
class dyadic(n_adic):
    
    def __init__(self,a:int,d:int,n_adic_reduce=True):
        super().__init__(a,2,d,n_adic_reduce)
        
    def conv_to_addr_list(self):
        current = self
        addr_list = []
        while current != dyadic(1,0):
            par = current + dyadic(1,current.exp)
            current_idx = current.exp - par.exp - 1
            addr_list.append(current_idx)
            current = par
        return list(reversed(addr_list))
    
    def __repr__(self):
        if self.exp > 10:
            return '{}/2^{}'.format(self.numerator,self.exp)
        return super().__repr__()

In [91]:
def conv_addr_list_to_dyadic(al):
    acc = dyadic(1,0)
    prev_exp = 0
    for i,x in enumerate(al):
        n_exp = x+prev_exp+1
        acc -= dyadic(1,n_exp)
        prev_exp = n_exp
    return acc


In [99]:
#quick test
max_exp = 100
for _ in range(10000):
    e = random.randint(1,max_exp)
    a = random.randint(0,(2**(e-1))-1)*2+1
    d_expected = dyadic(a,e)
    l_got = d_expected.conv_to_addr_list()
    d_got = conv_addr_list_to_dyadic(l_got)
    assert(d_got == d_expected)

Seems to be consistent at least

In [100]:
wack_list = [0,1,2,1,0,2,1,2,0,512,0,1,2,1]
len(encode_num_list(wack_list))

77

So as we're doing it now this gets us 77 bits. What if we encoded the section up to 512 as a dyadic, encoded 512 as a number in itself, and encoded the rest as a dyadic? This would give us the list:

In [106]:
[conv_addr_list_to_dyadic([0,1,2,1,0,2,1,2,0]),512,conv_addr_list_to_dyadic([0,1,2,1])]

[92589/262144, 512, 91/256]

Which can be encoded in 71 bits (likely not optimal, and changing the standard up a bit) as:

`0_1,16_7,5_3,46294_16,18_5, 1_1,10_5,0_1,0_1,512_10, 0_1,6_7,4_3,91_6,9_4`

In [191]:
def encode_dyadic(x,awidth_bits=7,ewidth_bits=3):
    #first width(a) and width(e)
    wa = w(x.numerator >> 1)
    we = w(x.exp)
    rst = tbw(wa,awidth_bits) + tbw(we,ewidth_bits)
    #then the numbers
    rst += tbw(x.numerator >> 1,wa) + tbw(x.exp,we)
    return rst

def decode_dyadic(rst,awidth_bits=7,ewidth_bits=3):
    wa = int(rst[:awidth_bits],base=2)
    rst = rst[awidth_bits:]
    we = int(rst[:ewidth_bits],base=2)
    rst = rst[ewidth_bits:]
    a = (int(rst[:wa],base=2) << 1) + 1
    rst = rst[wa:]
    e = int(rst[:we],base=2)
    rst = rst[we:]
    return dyadic(a,e),rst

In [139]:
x = dyadic(92589,18)
enc = encode_dyadic(x)
print(enc)
dec = decode_dyadic(enc)
print(dec)
assert(dec == x)

0010000101101101001101011010010
92589/2^18


In [140]:
len(encode_dyadic(x)),len(encode_num_list(x.conv_to_addr_list()))

(31, 41)

idea for an encoding standard:

```
1 bit: dyadic (0) block or RLE (1) block
if dyadic:
    7 bits: width(numerator) [w_a]
    3 bits: width(exp) [w_e]
    1 bit: repetition present
    w_a bits: numerator
    w_e bits: exp
    if repetition present:
        5 bits: nreps
if RLE:
    5 bits: number width (w_n)
    1 bit: repetition present (1) or not (0)
    1 bit: multiple numbers (1) or not (0)
    if multiple numbers:
        5 bits: length of sq width (w_l)
        w_l bits: length of run
    for each number in the sequence:
        w_n bits: the number
    if repetition present:
        5 bits: repetition width (w_r)
        w_r bits: nreps
repeat all until finished
```

this gives us 11 bits of overhead for each dyadic, 8 for each RLE (plus up to 10 more for a total of 18)

In [188]:
def encode_dyadic_with_repetition(x,rep,awidth_bits=7,ewidth_bits=3,rwidth_bits=5):
    rst = encode_dyadic(x,awidth_bits,ewidth_bits)
    rst = rst[:awidth_bits+ewidth_bits] + ('1' if rep > 1 else '0') + rst[awidth_bits+ewidth_bits:]
    if rep > 1:
        rst += tbw(rep,rwidth_bits)
    return rst

def decode_dyadic_with_repetition(cst,awidth_bits=7,ewidth_bits=3,rwidth_bits=5):
    #take out the repetition bit first
    rep_bit = cst[awidth_bits+ewidth_bits]
    cst = cst[:awidth_bits+ewidth_bits] + cst[awidth_bits+ewidth_bits+1:]
    #convert normally
    rl,cst = decode_dyadic(cst,awidth_bits,ewidth_bits)
    rl = rl.conv_to_addr_list()
    if rep_bit == '1':
        rep = int(cst[:rwidth_bits],base=2)
        cst = cst[rwidth_bits:]
    else:
        rep = 1
    #repeat
    while rep > 1:
        rl += rl.copy()
        rep -= 1
    return rl,cst

In [189]:
def RLE_encode_v2(l,rep,width_bits=5):
    num_w = w(max(l))
    rst = tbw(num_w,width_bits)
    rep_present = rep > 1
    rst += '1' if rep_present else '0'
    mul_nums = len(l) > 1
    rst += '1' if mul_nums else '0'
    if mul_nums:
        rst += tbw(w(len(l)),width_bits)
        rst += tbw(len(l),w(len(l)))
    for num in l:
        rst += tbw(num,num_w)
    if rep_present:
        rst += tbw(w(rep),width_bits)
        rst += tbw(rep,w(rep))
    return rst

def RLE_decode_v2(cst,width_bits=5):
    num_w = int(cst[:width_bits],base=2)
    cst = cst[width_bits:]
    rep_present = cst[0] == '1'
    mul_nums = cst[1] == '1'
    cst = cst[2:]
    if mul_nums:
        len_w = cst[:width_bits]
        cst = cst[width_bits:]
        num_ct = cst[:len_w]
        cst = cst[len_w:]
    else:
        num_ct = 1
    l = []
    for _ in range(num_ct):
        num_this = int(cst[:num_w],base=2)
        cst = cst[num_w:]
        l.append(num_this)
    if rep_present:
        rep_w = int(cst[:width_bits],base=2)
        cst = cst[width_bits:]
        rep = int(cst[:rep_w],base=2)
    else:
        rep = 1
    
    while rep > 1:
        l += l.copy()
        rep -= 1
    
    return l,cst
    

def hybrid_encode(l):
    #reduce the list
    lred = reduce_num_list(l)
    #unreduce the good bits
    lred = rrs_unformatted_all(lred)
    #encode each segment as dyadic and list, then pick the better one (lazy method)
    dys = [encode_dyadic_with_repetition(conv_addr_list_to_dyadic(lt),rep) for lt,rep in lred]
    rles = [RLE_encode_v2(lt,rep) for lt,rep in lred]
    rst = ''
    for dy,rle in zip(dys,rles):
        if len(dy) < len(rle):
            print('dy',dy)
            rst += '1'#this is a dyadic block
            rst += dy
        else:
            print('rle',rle)
            rst += '0'#this is an RLE block
            rst += rle
    return rst

def hybrid_decode(cst):
    l = []
    while len(cst) > 0:
        print(cst)
        dyadic_block = cst[0] == '1'
        cst = cst[1:]
        if dyadic_block:
            l_add,cst = decode_dyadic_with_repetition(cst)
            l += l_add
        else:
            l_add,cst = RLE_decode_v2(cst)
            l += l_add
    return l

In [197]:
len(hybrid_encode(wack_list))

dy 00100001010101101001101011010010
rle 01010001000000000
dy 000011010001011011000


73

### 4 bit save, woohoo!

In [192]:
decode_dyadic_with_repetition('00100001010101101001101011010010')

([0, 1, 2, 1, 0, 2, 1, 2, 0], '')

In [174]:
int('0101101',base=2)

45

In [195]:
hybrid_decode('1001000010101011010011010110100100010100010000000001000011010001011011000') == wack_list

1001000010101011010011010110100100010100010000000001000011010001011011000
0010100010000000001000011010001011011000
1000011010001011011000


True